In [20]:
import numpy as np
import pandas as pd

import re
import html
import requests

from datetime import datetime, date
import plotly.express as px
import plotly.graph_objs as go



def returncolname(row, colnames):
    if sum(row[['Q1', 'Q2','Q3','Q4']]) != 1:
        return colnames[np.argmax(row[['Q1_Prob', 'Q2_Prob','Q3_Prob','Q4_Prob']].values)]

def probabilityF(row):
    if row['colmax'] == 'Q1_Prob':
        return row.Q1_Prob
    elif row['colmax'] == 'Q2_Prob':
        return row.Q2_Prob
    elif row['colmax'] == 'Q3_Prob':
        return row.Q3_Prob
    elif row['colmax'] == 'Q4_Prob':
        return row.Q4_Prob

def Q1(row):
    if (row['Q1'] == 1.0) or (row['colmax'] == 'Q1_Prob'):
        return 1.0
    else:
        return 0.0
    
def Q2(row):
    if (row['Q2'] == 1.0) or (row['colmax'] == 'Q2_Prob'):
        return 1.0
    else:
        return 0.0

def Q3(row):
    if (row['Q3'] == 1.0) or (row['colmax'] == 'Q3_Prob'):
        return 1.0
    else:
        return 0.0
    
def Q4(row):
    if (row['Q4'] == 1.0) or (row['colmax'] == 'Q4_Prob'):
        return 1.0
    else:
        return 0.0
    
def q(row):
    if row['Q1 Posts'] == 1.0:
        return 'Q1'
    if row['Q2 Posts'] == 1.0:
        return 'Q2'
    if row['Q3 Posts'] == 1.0:
        return 'Q3'
    if row['Q4 Posts'] == 1.0:
        return 'Q4'
    else:
        return 'unk'

def buildout():
    """[Provides a final set of features showing which salary range applies to each data.]

    Returns:
        [csv]: [a table containing our original original, predicted, and bracket columns]
    """
    ### changed from app/
    df = pd.read_csv('../app/data/ml.csv')
    df['DatePosted'] = df['DatePosted'].astype('datetime64')
    df = df.sort_values(by="DatePosted")


    #df.replace(np.nan, 0, inplace = True)
    
    df['colmax'] = df.apply(lambda x: returncolname(x, df[['Q1_Prob', 'Q2_Prob','Q3_Prob','Q4_Prob']].columns), axis=1)
    df['Probability'] = df.apply(lambda x: probabilityF(x), axis=1)
    df['Q1 Posts'] = df.apply( lambda row : Q1(row), axis = 1)
    df['Q2 Posts'] = df.apply( lambda row : Q2(row), axis = 1)
    df['Q3 Posts'] = df.apply( lambda row : Q3(row), axis = 1)
    df['Q4 Posts'] = df.apply( lambda row : Q4(row), axis = 1)
    df['Q'] = df.apply( lambda row : q(row), axis = 1)

    
    df.Probability = df.Probability.fillna(1.0)
    # Saving final results table
    df.to_csv('../app/data/graph_ready.csv', index=False)
    
    # Creating and saving only observed salary data
    dfSal = df[df.Salary.notnull()]
    dfSal = dfSal[['DatePosted', 'Salary', 'Q']]
    dfSal.to_csv('../app/data/dfSal.csv', index=False)
    return df

In [21]:
df = buildout()

In [22]:
df.head(1)

,Company,JobTitle,Location,Requirements,Summary,Description,JobUrl,Salary,State,City,...,Q3_Prob,Q4_Pred,Q4_Prob,colmax,Probability,Q1 Posts,Q2 Posts,Q3 Posts,Q4 Posts,Q
10748,twitter,product data scientist - ads data science,"San Francisco, CA",none,2 years of work experience in data driven ana...,job descriptionwho we are the team is responsi...,https://www.indeed.com/rc/clk?jk=ca532d30df581...,NaN,CA,san francisco,...,1.0,0.0,0.0,Q3_Prob,1.0,0.0,0.0,1.0,0.0,Q3


In [23]:
from sklearn import metrics

In [24]:
df.columns

Index(['Company', 'JobTitle', 'Location', 'Requirements', 'Summary',
       'Description', 'JobUrl', 'Salary', 'State', 'City', 'DatePosted',
       'Schedule', 'Role', 'Focus', 'Q1', 'Q2', 'Q3', 'Q4', 'Q1_Pred',
       'Q1_Prob', 'Q2_Pred', 'Q2_Prob', 'Q3_Pred', 'Q3_Prob', 'Q4_Pred',
       'Q4_Prob', 'colmax', 'Probability', 'Q1 Posts', 'Q2 Posts', 'Q3 Posts',
       'Q4 Posts', 'Q'],
      dtype='object')

In [25]:
dfc = df[df.Salary.notnull()]

In [26]:
dfProbs = dfc[['Q1', 'Q2', 'Q3', 'Q4',
       'Q1_Pred', 'Q1_Prob', 'Q2_Pred', 'Q2_Prob', 'Q3_Pred', 'Q3_Prob',
       'Q4_Pred', 'Q4_Prob','Salary']]
dfProbs

,Q1,Q2,Q3,Q4,Q1_Pred,Q1_Prob,Q2_Pred,Q2_Prob,Q3_Pred,Q3_Prob,Q4_Pred,Q4_Prob,Salary
24074,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177500.000000
24077,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,135571.428571
24083,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180000.000000
24061,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114660.000000
24035,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121325.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25344,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31285.714286
25345,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90803.000000
25346,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62000.000000
25347,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156428.571429


In [27]:
df2 = df[['Q1_Pred', 'Q2_Pred', 'Q3_Pred', 'Q4_Pred']]

In [28]:
df2

,Q1_Pred,Q2_Pred,Q3_Pred,Q4_Pred
10748,0.0,0.0,1.0,0.0
10381,0.0,0.0,0.0,0.0
10679,0.0,0.0,0.0,0.0
10684,0.0,1.0,1.0,0.0
10685,0.0,0.0,0.0,0.0
...,...,...,...,...
22590,0.0,0.0,0.0,1.0
22591,0.0,1.0,1.0,0.0
22592,1.0,1.0,0.0,0.0
22574,0.0,0.0,0.0,0.0


In [29]:
df.columns

Index(['Company', 'JobTitle', 'Location', 'Requirements', 'Summary',
       'Description', 'JobUrl', 'Salary', 'State', 'City', 'DatePosted',
       'Schedule', 'Role', 'Focus', 'Q1', 'Q2', 'Q3', 'Q4', 'Q1_Pred',
       'Q1_Prob', 'Q2_Pred', 'Q2_Prob', 'Q3_Pred', 'Q3_Prob', 'Q4_Pred',
       'Q4_Prob', 'colmax', 'Probability', 'Q1 Posts', 'Q2 Posts', 'Q3 Posts',
       'Q4 Posts', 'Q'],
      dtype='object')

In [30]:
df[['JobTitle', 'Company', 'Location', 'Summary', 'JobUrl', 'Requirements',
       'Description', 'State', 'DatePosted','Salary','Probability','Q'] ].reset_index()

,index,JobTitle,Company,Location,Summary,JobUrl,Requirements,Description,State,DatePosted,Salary,Probability,Q
0,10748,product data scientist - ads data science,twitter,"San Francisco, CA",2 years of work experience in data driven ana...,https://www.indeed.com/rc/clk?jk=ca532d30df581...,none,job descriptionwho we are the team is responsi...,CA,2021-04-14,NaN,1.00,Q3
1,10381,"data scientist, operations decision science",delta,"Atlanta, GA",an appropriate candidate will have a proven ex...,https://www.indeed.com/rc/clk?jk=1cb59dd5ec8ba...,none,the operations analysis performance oap te...,GA,2021-04-14,NaN,0.32,Q4
2,10679,data scientist,commerce bank,"Kansas City, MO",1 2 years statistical analysis machine learni...,https://www.indeed.com/rc/clk?jk=68c23b12100e4...,none,about working at commercewouldn’t it be great ...,MO,2021-04-14,NaN,0.32,Q4
3,10684,data scientist - customer service,dte energy,"Detroit, MI",experience in quantitative analytics e g dat...,https://www.indeed.com/rc/clk?jk=caf39a8e478b2...,none,the future is bright at dte energy we are one...,MI,2021-04-14,NaN,0.99,Q3
4,10685,associate data scientist,catalina marketing,"Chicago, IL",research and test new measurement and targetin...,https://www.indeed.com/rc/clk?jk=cc3a61acd7a7c...,none,the digital analytics team at catalina is resp...,IL,2021-04-14,NaN,0.00,Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25391,22590,sr mgr marketing analytics - remote,"thryv, inc.",United States•Remote,must be 18 years of age or older you will be ...,https://www.indeed.com/rc/clk?jk=50c2fe5a216f9...,none,as sr manager marketing analytics you will lea...,Remote,2021-08-17,NaN,0.98,Q4
25392,22591,corporate counsel,coursera,Remote,legal and compliance is responsible for managi...,https://www.indeed.com/rc/clk?jk=e3042ccfbf8bf...,none,coursera was launched in 2012 by two stanford ...,Remote,2021-08-17,NaN,0.99,Q2
25393,22592,biostatistician,avera,"Sioux Falls, SD 57105",masters degree or phd in biostatistics statis...,https://www.indeed.com/rc/clk?jk=7078273ad6368...,none,job summary the overall function of this role...,SD,2021-08-17,NaN,1.00,Q1
25394,22574,data scientist - product analytics,match group,"Dallas, TX",you have at least 2 years of professional expe...,https://www.indeed.com/rc/clk?jk=006e78a03c017...,none,match boasts more dates more relationships an...,TX,2021-08-17,NaN,0.32,Q4


In [31]:
a = df[df.Salary.isnull()]

In [32]:
b= df[df.Salary.notnull()]

In [33]:
df[df.Probability.isnull()]

,Company,JobTitle,Location,Requirements,Summary,Description,JobUrl,Salary,State,City,...,Q3_Prob,Q4_Pred,Q4_Prob,colmax,Probability,Q1 Posts,Q2 Posts,Q3 Posts,Q4 Posts,Q


In [34]:
z = pd.concat([a,b])

In [35]:
za = z[['Salary', 'Company', 'Location', 'Summary', 'JobUrl', 'Requirements',
       'Description', 'State', 'DatePosted','Probability','Q']]
zb = z[['Salary', 'Q1', 'Q2', 'Q3', 'Q4', 'Q1_Pred',
       'Q1_Prob', 'Q2_Pred', 'Q2_Prob', 'Q3_Pred', 'Q3_Prob', 'Q4_Pred',
       'Q4_Prob', 'colmax', 'Q1 Posts', 'Q2 Posts', 'Q3 Posts',
       'Q4 Posts', 'Probability', 'Q']]

In [36]:
za

,Salary,Company,Location,Summary,JobUrl,Requirements,Description,State,DatePosted,Probability,Q
10748,NaN,twitter,"San Francisco, CA",2 years of work experience in data driven ana...,https://www.indeed.com/rc/clk?jk=ca532d30df581...,none,job descriptionwho we are the team is responsi...,CA,2021-04-14,1.00,Q3
10381,NaN,delta,"Atlanta, GA",an appropriate candidate will have a proven ex...,https://www.indeed.com/rc/clk?jk=1cb59dd5ec8ba...,none,the operations analysis performance oap te...,GA,2021-04-14,0.32,Q4
10679,NaN,commerce bank,"Kansas City, MO",1 2 years statistical analysis machine learni...,https://www.indeed.com/rc/clk?jk=68c23b12100e4...,none,about working at commercewouldn’t it be great ...,MO,2021-04-14,0.32,Q4
10684,NaN,dte energy,"Detroit, MI",experience in quantitative analytics e g dat...,https://www.indeed.com/rc/clk?jk=caf39a8e478b2...,none,the future is bright at dte energy we are one...,MI,2021-04-14,0.99,Q3
10685,NaN,catalina marketing,"Chicago, IL",research and test new measurement and targetin...,https://www.indeed.com/rc/clk?jk=cc3a61acd7a7c...,none,the digital analytics team at catalina is resp...,IL,2021-04-14,0.00,Q1
...,...,...,...,...,...,...,...,...,...,...,...
25344,31285.714286,rhythm,"Katy, TX",high school diploma or ged certificate high s...,https://www.indeed.com/company/Rhythm/jobs/Ent...,high school or equivalent (preferred)sales exp...,rhythm is a renewable energy and technology co...,TX,2021-08-17,1.00,Q1
25345,90803.000000,equal employ practices comm,"Manhattan, NY",for assignment level i only physical biologi...,https://www.indeed.com/rc/clk?jk=983ec8e54f1ca...,none,the equal employment practices commission emp...,NY,2021-08-17,1.00,Q2
25346,62000.000000,veterans research and education foundation of ...,"St. Louis, MO 63103",experience managing or utilizing data warehous...,https://www.indeed.com/company/Veterans-Resear...,master's (required)us work authorization (requ...,biostatistician programmer data scientist the ...,MO,2021-08-17,1.00,Q1
25347,156428.571429,yellow jacket inc,"Boston, MA",6 8 yrs of experience working in advanced ana...,https://www.indeed.com/company/Yellow-Jacket-I...,sql: 5 years (preferred)advanced analytics: 5 ...,key responsibilities· partner with internal st...,MA,2021-08-17,1.00,Q4


In [37]:
zb

,Salary,Q1,Q2,Q3,Q4,Q1_Pred,Q1_Prob,Q2_Pred,Q2_Prob,Q3_Pred,Q3_Prob,Q4_Pred,Q4_Prob,colmax,Q1 Posts,Q2 Posts,Q3 Posts,Q4 Posts,Probability,Q
10748,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.00,1.0,1.00,0.0,0.00,Q3_Prob,0.0,0.0,1.0,0.0,1.00,Q3
10381,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.32,Q4_Prob,0.0,0.0,0.0,1.0,0.32,Q4
10679,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.00,0.0,0.04,0.0,0.32,Q4_Prob,0.0,0.0,0.0,1.0,0.32,Q4
10684,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.92,1.0,0.99,0.0,0.00,Q3_Prob,0.0,0.0,1.0,0.0,0.99,Q3
10685,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,Q1_Prob,1.0,0.0,0.0,0.0,0.00,Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25344,31285.714286,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1.0,0.0,0.0,0.0,1.00,Q1
25345,90803.000000,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,1.0,0.0,0.0,1.00,Q2
25346,62000.000000,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1.0,0.0,0.0,0.0,1.00,Q1
25347,156428.571429,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0.0,0.0,0.0,1.0,1.00,Q4


In [19]:
c = df2[df2.Salary.isnull()]

AttributeError: 'DataFrame' object has no attribute 'Salary'

In [ ]:
d = df2[df2.Salary.notnull()]

In [ ]:
zz = pd.concat([c,d])
zz

In [ ]:
a = sum(df['Q1 Posts'] ==1)
a

In [ ]:
b = sum(df['Q2 Posts'] ==1)
b

In [ ]:
c = sum(df['Q3 Posts'] ==1)
c

In [ ]:
d = sum(df['Q4 Posts'] ==1)
d

In [ ]:
sum([a,b,c,d])

In [ ]:
dd = pd.read_csv('../app/data/ml.csv')

In [ ]:
dd.columns

In [ ]:
dd